In [1]:
import pdfplumber
import pytesseract
from PIL import Image       # pdfplumber usa PIL internamente
import pandas as pd

In [4]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [2]:
def extrair_conteudo_pdf(caminho_pdf):
    """Extrai texto, imagens (via OCR) e tabelas de um PDF em uma única string."""
    conteudo = []  # lista de segmentos de texto extraídos

    with pdfplumber.open(caminho_pdf) as pdf:
        for num_pagina, pagina in enumerate(pdf.pages, start=1):
            # Extrai texto da página (se houver)
            texto = pagina.extract_text()
            if texto:
                conteudo.append(texto)
            
            # Extrai tabelas da página (se alguma)
            tabelas = pagina.extract_tables()
            for tabela in tabelas:
                if tabela:
                    # Converte lista de linhas em DataFrame pandas para formatação
                    # A primeira linha da tabela pode ser cabeçalho (usada aqui como nomes de coluna)
                    df = pd.DataFrame(tabela[1:], columns=tabela[0] if tabela[0] else None)
                    # Converte a tabela em string (formato tabular) e adiciona
                    conteudo.append(df.to_string(index=False))
            
            # Extrai imagens da página e aplica OCR
            for img_meta in pagina.images:
                # Obtém coordenadas da imagem no PDF
                x0 = img_meta["x0"]
                top = img_meta["top"]
                x1 = img_meta["x1"]
                bottom = img_meta["bottom"]
                # Recorta a região da imagem na página
                recorte = pagina.crop((x0, top, x1, bottom))
                # Converte o recorte em uma imagem PIL (aumentar resolução para melhorar OCR se necessário)
                pil_img = recorte.to_image(resolution=150).original
                # Aplica OCR na imagem recortada
                texto_img = pytesseract.image_to_string(pil_img, lang="por")  # lang="por" para OCR em português, se aplicável
                texto_img = texto_img.strip()
                if texto_img:
                    # Opcional: indicar no texto que era conteúdo de imagem ou posição original
                    conteudo.append(f"[Imagem na página {num_pagina}]\n{texto_img}")
    
    # Junta todos os pedaços em uma única string, separando por quebras de linha entre partes
    return "\n\n".join(conteudo)


In [5]:
resultado = extrair_conteudo_pdf("Equacao_do_Segundo_Grau.pdf")


In [6]:
print(resultado)

Equação do Segundo Grau
Denomina-se equação do 2° grau, qualquer sentença matemática que possa ser reduzida à forma
ax2 + bx + c = 0, onde x é a incógnita e a, b e c são números reais, com a ≠ 0. a, b e c são coeficientes da
equação. Observe que o maior índice da incógnita na equação é igual a dois e é isto que a define como sendo
uma equação do segundo grau.
Equação do 2° grau completa e equação do 2° grau incompleta
Da definição acima temos obrigatoriamente que a ≠ 0, no entanto podemos ter b = 0 e/ou c = 0.
Caso b ≠ 0 e c ≠ 0, temos uma equação do 2° grau completa. A sentença matemática -2x2 + 3x - 5 = 0 é um
exemplo de equação do 2° grau completa, pois temos b = 3 e c = -5, que são diferentes de zero.
-x2 + 7 = 0 é um exemplo de equação do 2° grau incompleta, pois b = 0.
Neste outro exemplo, 3x2 - 4x = 0 a equação é incompleta, pois c = 0.
Veja este último exemplo de equação do 2° grau incompleta, 8x2 = 0, onde tanto b, quanto c são iguais a
zero.
Resolução de equações do 2° grau
A